In [1]:
from dataset import MolerDataset, MolerData
from utils import pprint_pyg_obj
from torch_geometric.loader import DataLoader

dataset = MolerDataset(
    root = '/data/ongh0068', 
    raw_moler_trace_dataset_parent_folder = '/data/ongh0068/l1000/trace_playground',
    output_pyg_trace_dataset_parent_folder = '/data/ongh0068/l1000/pyg_output_playground',
    split = 'train',
)

loader = DataLoader(dataset, batch_size=16, shuffle=False, follow_batch = [
    'correct_edge_choices',
    'correct_edge_types',
    'valid_edge_choices',
    'valid_attachment_point_choices',
    'correct_attachment_point_choice',
    'correct_node_type_choices',
    'original_graph_x',
    'correct_first_node_type_choices'
])

2022-12-21 12:57:39.218564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
for batch in loader:
    break

In [8]:
import torch
params = {'full_graph_encoder': {'input_feature_dim': 32,
  'atom_or_motif_vocab_size': 139},
 'partial_graph_encoder': {'input_feature_dim': 32},
 'mean_log_var_mlp': {'input_feature_dim': 832, 'output_size': 1024},
 'decoder': {'node_type_selector': {'input_feature_dim': 1344,
   'output_size': 140},
  'node_type_loss_weights': torch.tensor([10.0000,  0.1000,  0.1000,  0.1000,  0.7879,  0.4924,  0.6060, 10.0000,
           7.8786, 10.0000,  7.8786,  0.1000,  0.6565,  0.6565,  0.9848,  0.8754,
           0.8754,  1.1255,  0.9848,  1.3131,  1.5757,  1.9696,  1.5757,  1.9696,
           2.6262,  1.9696,  1.9696,  7.8786,  7.8786,  3.9393,  2.6262,  2.6262,
           2.6262,  2.6262,  3.9393,  7.8786,  7.8786,  7.8786,  3.9393,  7.8786,
          10.0000,  7.8786,  3.9393,  3.9393,  3.9393,  3.9393,  3.9393,  3.9393,
           3.9393,  3.9393,  3.9393,  3.9393,  3.9393,  3.9393,  7.8786,  7.8786,
          10.0000, 10.0000,  7.8786,  7.8786, 10.0000,  7.8786,  7.8786, 10.0000,
           7.8786,  7.8786, 10.0000,  7.8786, 10.0000,  7.8786,  7.8786, 10.0000,
           7.8786,  7.8786,  7.8786, 10.0000, 10.0000,  7.8786,  7.8786,  7.8786,
           7.8786,  7.8786, 10.0000, 10.0000, 10.0000, 10.0000,  7.8786, 10.0000,
          10.0000, 10.0000,  7.8786, 10.0000,  7.8786, 10.0000,  7.8786, 10.0000,
          10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000,  7.8786,
          10.0000,  7.8786,  7.8786,  7.8786,  7.8786, 10.0000,  7.8786, 10.0000,
          10.0000, 10.0000,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,
           7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,
           7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,  7.8786,
           7.8786,  7.8786,  7.8786,  0.1000]),
  'no_more_edges_repr': (1, 835),
  'edge_candidate_scorer': {'input_feature_dim': 3011, 'output_size': 1},
  'edge_type_selector': {'input_feature_dim': 3011, 'output_size': 3},
  'attachment_point_selector': {'input_feature_dim': 2176, 'output_size': 1},
  'first_node_type_selector': {'input_feature_dim': 512, 'output_size': 139}},
 'latent_sample_strategy': 'per_graph',
 'latent_repr_dim': 512,
 'latent_repr_size': 512}

In [9]:
%load_ext autoreload
%autoreload 2
from model import BaseModel


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
model = BaseModel(params, dataset).eval()

moler_output = model._run_step(batch)

loss = model.compute_loss(moler_output, batch)

loss

tensor(5.8672, grad_fn=<AddBackward0>)

In [6]:
def decode(
    graph_representations,
    initial_molecules= None,
    mol_ids = None,
    store_generation_traces = False,
    max_num_steps=120,
    beam_size= 1,
    sampling_mode = 'greedy',
)

SyntaxError: invalid syntax (2536383891.py, line 9)

In [42]:
with torch.no_grad():
    input_molecule_representations = model._full_graph_encoder(
        original_graph_node_categorical_features=batch.original_graph_node_categorical_features,
        node_features=batch.original_graph_x.float(),
        edge_index=batch.original_graph_edge_index,
        edge_type=batch.original_graph_edge_type.int(),
        batch_index=batch.original_graph_x_batch,
    )


    partial_graph_representions, node_representations = model._partial_graph_encoder(
        node_features=batch.x,
        edge_index=batch.edge_index.long(),
        edge_type=batch.edge_type.int(),
        batch_index=batch.batch,
    )
    # Apply latent sampling strategy
    p, q, latent_representations = model.sample_from_latent_repr(
        input_molecule_representations
    )

In [13]:
latent_representation.shape

torch.Size([16, 512])

In [14]:
from rdkit import Chem
from molecule_generation.chem.molecule_dataset_utils import BOND_DICT
from molecule_generation.chem.motif_utils import (
    find_motifs_from_vocabulary,
)
uses_motifs = True
initial_molecules = None
mol_ids = None
store_generation_traces = False

graph_representations = latent_representation

if initial_molecules is None:
    initial_molecules = [None] * len(graph_representations)

# Replace `None` in initial_molecules with empty molecules.
initial_molecules = [
    Chem.Mol() if initMol is None else initMol for initMol in initial_molecules
]
if mol_ids is None:
    mol_ids = range(len(graph_representations))

In [15]:
from molecule_generation.utils.moler_decoding_utils import (
    DecoderSamplingMode,
    sample_indices_from_logprobs,
    restrict_to_beam_size_per_mol,
    MoLeRDecoderState
)

In [18]:
decoder_states = []


# preprocessing for when a scaffold is given
for graph_repr, init_mol, mol_id in zip(graph_representations, initial_molecules, mol_ids):
    num_free_bond_slots = [0] * len(init_mol.GetAtoms())

    atom_ids_to_remove = []
    atom_ids_to_keep = []

    for atom in init_mol.GetAtoms():
        if atom.GetAtomicNum() == 0:
            # Atomic number 0 means a placeholder atom that signifies an attachment point.
            bonds = atom.GetBonds()

            if len(bonds) > 1:
                scaffold = Chem.MolToSmiles(init_mol)
                raise ValueError(
                    f"Scaffold {scaffold} contains a [*] atom with at least two bonds."
                )

            if not bonds:
                # This is a very odd case: either the scaffold we got is disconnected, or
                # it consists of just a single * atom.
                scaffold = Chem.MolToSmiles(init_mol)
                raise ValueError(f"Scaffold {scaffold} contains a [*] atom with no bonds.")

            [bond] = bonds
            begin_idx = bond.GetBeginAtomIdx()
            end_idx = bond.GetEndAtomIdx()

            neighbour_idx = begin_idx if begin_idx != atom.GetIdx() else end_idx
            num_free_bond_slots[neighbour_idx] += 1

            atom_ids_to_remove.append(atom.GetIdx())
        else:
            atom_ids_to_keep.append(atom.GetIdx())

    if not atom_ids_to_remove:
        # No explicit attachment points, so assume we can connect anywhere.
        num_free_bond_slots = None
    else:
        num_free_bond_slots = [num_free_bond_slots[idx] for idx in atom_ids_to_keep]
        init_mol = Chem.RWMol(init_mol)

        # Remove atoms starting from largest index, so that we don't have to account for
        # indices shifting during removal.
        for atom_idx in reversed(atom_ids_to_remove):
            init_mol.RemoveAtom(atom_idx)

        # Determine how the scaffold atoms will get reordered when we canonicalize it, so we can
        # permute `num_free_bond_slots` appropriately.
        canonical_ordering = compute_canonical_atom_order(init_mol)
        num_free_bond_slots = [num_free_bond_slots[idx] for idx in canonical_ordering]

    # Now canonicalize, which renumbers all the atoms, but we've applied the same
    # renumbering to `num_free_bond_slots` earlier.
    init_mol = Chem.MolFromSmiles(Chem.MolToSmiles(init_mol))

    # Clear aromatic flags in the scaffold, since partial graphs during training never have
    # them set (however we _do_ run `AtomIsAromaticFeatureExtractor`, it just always returns
    # 0 for partial graphs during training).
    # TODO(kmaziarz): Consider fixing this.
    Chem.Kekulize(init_mol, clearAromaticFlags=True)

    init_atom_types = []
    # TODO(kmaziarz): We need to be more careful in how the initial molecule looks like, to
    # make sure that `init_mol`s have correct atom features (e.g. charges).
    for atom in init_mol.GetAtoms():
        init_atom_types.append(get_atom_symbol(atom))
    adjacency_lists = [[] for _ in range(len(BOND_DICT))]
    for bond in init_mol.GetBonds():
        bond_type_idx = BOND_DICT[str(bond.GetBondType())]
        adjacency_lists[bond_type_idx].append(
            (bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())
        )
        adjacency_lists[bond_type_idx].append(
            (bond.GetEndAtomIdx(), bond.GetBeginAtomIdx())
        )

    if uses_motifs:
        init_mol_motifs = find_motifs_from_vocabulary(
            molecule=init_mol, motif_vocabulary=dataset._motif_vocabulary
        )
    else:
        init_mol_motifs = []

    decoder_states.append(
        MoLeRDecoderState(
            molecule_representation=graph_repr,
            molecule_id=mol_id,
            molecule=init_mol,
            atom_types=init_atom_types,
            adjacency_lists=adjacency_lists,
            visited_atoms=[atom.GetIdx() for atom in init_mol.GetAtoms()],
            atoms_to_visit=[],
            focus_atom=None,
            # Pseudo-randomly pick last atom from input:
            prior_focus_atom=len(init_atom_types) - 1,
            generation_steps=[] if store_generation_traces else None,
            motifs=init_mol_motifs,
            num_free_bond_slots=num_free_bond_slots,
        )
    )

decoder_states_empty = []
decoder_states_non_empty = []

for decoder_state in decoder_states:
    if decoder_state.molecule.GetNumAtoms() == 0:
        decoder_states_empty.append(decoder_state)
    else:
        decoder_states_non_empty.append(decoder_state)


# TODO


In [25]:
from decoder import MLPDecoder
from model_utils import get_params

decoder = MLPDecoder(get_params()['decoder'])

In [37]:
dataset.node_type_index_to_string

{0: 'UNK',
 1: 'C',
 2: 'O',
 3: 'N',
 4: 'S',
 5: 'Cl',
 6: 'F',
 7: 'N+',
 8: 'O-',
 9: 'B',
 10: 'Br',
 11: 'C1=CC=CC=C1',
 12: 'C1CCNCC1',
 13: 'C1=CC=NC=C1',
 14: 'NC=O',
 15: 'C1CNCCN1',
 16: 'C1CCNC1',
 17: 'O=[N+][O-]',
 18: 'C1CC1',
 19: 'C1COCCN1',
 20: 'O=CO',
 21: 'N=CO',
 22: 'C1=CNN=C1',
 23: 'FC(F)F',
 24: 'CC=O',
 25: 'C1CCOCC1',
 26: 'C1=CC=C2C=CC=CC2=C1',
 27: 'COC=O',
 28: 'CCO',
 29: 'C=CC',
 30: 'C1=CSC=N1',
 31: 'C1=CSC=C1',
 32: 'C1=CON=C1',
 33: 'C1=CC=C2NC=CC2=C1',
 34: 'OC(F)F',
 35: 'O=CNO',
 36: 'CNC=O',
 37: 'CCNC(=O)OC',
 38: 'CCCO',
 39: 'CCC(N)=O',
 40: 'CCC(=O)O',
 41: 'CC(N)=O',
 42: 'CC(C)C',
 43: 'C1CCOC1',
 44: 'C1CCCCC1',
 45: 'C1=NC=C2CCCCC2=N1',
 46: 'C1=COC=C1',
 47: 'C1=CNC=C1',
 48: 'C1=CN=CN=C1',
 49: 'C1=CN=C2C=CC=CC2=C1',
 50: 'C1=CN2N=CC=C2N=C1',
 51: 'C1=CC=C2OC=CCC2=C1',
 52: 'C1=CC=C2N=CC=CC2=C1',
 53: 'C1=CC2=C(CCC2)S1',
 54: 'O=S=O',
 55: 'O=CNCCO',
 56: 'O=CNCCCCCNC=O',
 57: 'O=CNCCC(=O)O',
 58: 'O=CCCO',
 59: 'O=C(O)CS',
 60: 'N[SH]

In [28]:
decoder._first_node_type_selector(latent_representations).shape

torch.Size([16, 139])

In [49]:
from decoding_utils import sample_indices_from_logprobs
import numpy as np


num_samples = 1
decoder_states = decoder_states_empty
sampling_mode =  'greedy'
index_to_node_type_map=dataset.node_type_index_to_string

with torch.no_grad():
    # We only need the molecule representations.
    molecule_representations = torch.stack(
        [state.molecule_representation for state in decoder_states]
    )

    first_node_type_logits = model.decoder.pick_first_node_type(
        latent_representations=latent_representations
    )  # Shape [G, NT + 1]

    first_atom_type_logprobs = torch.nn.functional.log_softmax(
        first_node_type_logits[:, 1:], dim=-1 # because index 0 corresponds to UNK
    )  # Shape [G, NT]

    first_atom_type_pick_results = []


    # Iterate over each of the rows independently, sampling for each input state:
    for state_first_atom_type_logprobs in first_atom_type_logprobs:
        picked_atom_type_indices = sample_indices_from_logprobs(
            num_samples, sampling_mode, state_first_atom_type_logprobs
        )

        this_state_results = []

        for picked_atom_type_idx in picked_atom_type_indices:
            pick_logprob = state_first_atom_type_logprobs[picked_atom_type_idx]
            picked_atom_type_idx += 1  # Revert the stripping out of the UNK (index 0) type

            this_state_results.append(
                (index_to_node_type_map[picked_atom_type_idx.item()], pick_logprob)
            )

        first_atom_type_pick_results.append(
            (this_state_results, state_first_atom_type_logprobs)
        )


In [50]:
first_atom_type_pick_results

[([('C1=CSC=N1', tensor(-4.4922))],
  tensor([-4.9866, -5.0678, -4.7991, -4.9423, -4.8531, -4.9125, -5.0838, -4.9868,
          -4.8487, -4.8871, -4.8654, -4.7842, -4.8416, -5.1022, -4.6453, -5.0501,
          -5.1004, -4.9114, -4.8446, -5.2496, -4.9498, -5.1721, -4.9902, -5.0658,
          -4.6634, -4.8596, -4.9049, -5.3538, -4.9552, -4.4922, -5.3117, -4.9221,
          -5.0452, -4.9354, -4.8987, -5.1605, -5.0775, -4.9880, -5.0293, -5.1482,
          -4.8643, -4.9973, -4.8408, -5.0244, -5.0430, -4.9576, -4.9309, -5.1348,
          -4.9058, -4.8333, -4.8235, -4.8025, -4.9150, -5.0250, -5.0331, -5.2307,
          -4.8528, -4.8655, -4.8332, -5.0042, -4.7533, -4.9817, -5.0698, -4.7578,
          -4.8405, -5.0142, -4.7864, -4.9067, -4.9494, -4.8267, -4.5806, -4.8468,
          -4.8951, -4.8767, -4.8059, -5.1099, -4.9904, -5.1228, -4.7057, -4.8008,
          -4.8669, -4.8698, -4.9531, -4.9734, -4.7531, -4.8015, -5.0493, -4.8709,
          -4.8768, -4.7583, -4.8515, -5.0591, -4.7950, -4.8158

In [40]:
molecule_representations.shape

torch.Size([16, 512])

In [22]:
def _decoder_pick_first_atom_types(
    self,
    *,
    decoder_states: List[MoLeRDecoderState],
    sampling_mode: DecoderSamplingMode,
    num_samples: int = 1,
) -> Iterable[Tuple[List[Tuple[Optional[str], float]], np.ndarray]]:
    """
    Query the model to pick the first atom to add for each of a list of decoder states.

    Args:
        decoder_states: MoLeRDecoderState objects representing partial
            results of the decoder that are to be extended by an additional atom.
        sampling_mode: Determines how to obtain num_samples. GREEDY takes the most
            likely values, whereas SAMPLING samples according to the predicted
            probabilities.
        num_samples: Number of samples to return per input decoder state (non-1 values
            are useful for beam search).

    Returns:
        A list of the same length as the input list `decoder_states`; the i-th entry
        is the result for the i-th input.
        A single result contains a list of `num_samples` first node type choices, and
        an array containing all the first node type logprobs.
    """
    if len(decoder_states) == 0:
        return []

    if not self.has_first_node_type_selector:
        # Models without the first node type selector always start decoding from carbon.
        def generate_result_that_picks_carbon():
            first_node_type_picks = [("C", 0.0) for _ in range(num_samples)]

            # Probability of selecting anything other than carbon is 0.
            first_node_type_logprobs = np.full(self._num_node_types, -BIG_NUMBER)
            first_node_type_logprobs[self._carbon_type_idx] = 0.0

            return first_node_type_picks, first_node_type_logprobs

        return [generate_result_that_picks_carbon() for _ in range(len(decoder_states))]

    # We only need the molecule representations.
    molecule_representations = np.stack(
        [state.molecule_representation for state in decoder_states]
    )

    first_node_type_logits = self.pick_first_node_type(
        input_molecule_representations=molecule_representations, training=False
    )  # Shape [G, NT + 1]

    first_atom_type_logprobs = tf.nn.log_softmax(
        first_node_type_logits[:, 1:], axis=1
    ).numpy()  # Shape [G, NT]

    first_atom_type_pick_results: List[
        Tuple[List[Tuple[Optional[str], float]], np.ndarray]
    ] = []

    # Iterate over each of the rows independently, sampling for each input state:
    for state_first_atom_type_logprobs in first_atom_type_logprobs:
        picked_atom_type_indices = sample_indices_from_logprobs(
            num_samples, sampling_mode, state_first_atom_type_logprobs
        )

        this_state_results: List[Tuple[Optional[str], float]] = []

        for picked_atom_type_idx in picked_atom_type_indices:
            pick_logprob: float = state_first_atom_type_logprobs[picked_atom_type_idx]
            picked_atom_type_idx += 1  # Revert the stripping out of the UNK (index 0) type

            this_state_results.append(
                (self._index_to_node_type_map[picked_atom_type_idx], pick_logprob)
            )

        first_atom_type_pick_results.append(
            (this_state_results, state_first_atom_type_logprobs)
        )

    return first_atom_type_pick_results

NameError: name 'List' is not defined

In [ ]:

# Step 0: Pick first node types for states that do not have an initial molecule.
first_node_pick_results = self._decoder_pick_first_atom_types(
    decoder_states=decoder_states_empty, num_samples=beam_size, sampling_mode=sampling_mode
)

# print("I: Picked first node types:", [picks for picks, _ in first_node_pick_results])

decoder_states = decoder_states_non_empty

for decoder_state, (first_node_type_picks, first_node_type_logprobs) in zip(
    decoder_states_empty, first_node_pick_results
):
    for first_node_type_pick, first_node_type_logprob in first_node_type_picks:
        # Set up generation trace storing variables, populating if needed.
        atom_choice_info = None
        if store_generation_traces:
            atom_choice_info = MoleculeGenerationAtomChoiceInfo(
                node_idx=0,
                true_type_idx=None,
                type_idx_to_prob=np.exp(first_node_type_logprobs),
            )

        new_decoder_state, added_motif = self._add_atom_or_motif(
            decoder_state,
            first_node_type_pick,
            logprob=first_node_type_logprob,
            choice_info=atom_choice_info,
        )

        last_atom_id = new_decoder_state.molecule.GetNumAtoms() - 1

        if added_motif:
            # To make all asserts happy, pretend we chose an attachment point.
            new_decoder_state._focus_atom = last_atom_id

        # Mark all initial nodes as visited.
        new_decoder_state = MoLeRDecoderState.new_with_focus_marked_as_visited(
            old_state=new_decoder_state, focus_node_finished_logprob=0.0
        )

        # Set the prior focus atom similarly to the start-from-scaffold case.
        new_decoder_state._prior_focus_atom = last_atom_id

        decoder_states.append(new_decoder_state)


In [ ]:

num_steps = 0
while num_steps < max_num_steps:
    # This will hold the results after this decoding step, grouped by input mol id:
    new_decoder_states: List[MoLeRDecoderState] = []
    num_steps += 1
    # Step 1: Split decoder states into subsets, dependent on what they need next:
    require_atom_states, require_bond_states, require_attachment_point_states = [], [], []
    for decoder_state in decoder_states:
        # No focus atom => needs a new atom
        if decoder_state.focus_atom is None:
            require_atom_states.append(decoder_state)
        # Focus atom has invalid index => decoding finished, just push forward unchanged:
        elif decoder_state.focus_atom < 0:
            new_decoder_states.append(decoder_state)
        else:
            require_bond_states.append(decoder_state)

    # Check if we are done:
    if (len(require_atom_states) + len(require_bond_states)) == 0:
        # print("I: Decoding finished")
        break

    # Step 2: For states that require a new atom, try to pick one:
    node_pick_results = self._decoder_pick_new_atom_types(
        decoder_states=require_atom_states,
        num_samples=beam_size,
        sampling_mode=sampling_mode,
    )

    for decoder_state, (node_type_picks, node_type_logprobs) in zip(
        require_atom_states, node_pick_results
    ):
        for node_type_pick, node_type_logprob in node_type_picks:
            # Set up generation trace storing variables, populating if needed.
            atom_choice_info = None
            if store_generation_traces:
                atom_choice_info = MoleculeGenerationAtomChoiceInfo(
                    node_idx=decoder_state.prior_focus_atom + 1,
                    true_type_idx=None,
                    type_idx_to_prob=np.exp(node_type_logprobs),
                )

            # If the decoder says we need no new atoms anymore, we are finished. Otherwise,
            # start adding more bonds:
            if node_type_pick is None:
                # print(I {decoder_state.molecule_id} {decoder_state.logprob:12f}: Finished decoding - p={node_type_logprob:5f}")
                new_decoder_states.append(
                    MoLeRDecoderState.new_for_finished_decoding(
                        old_state=decoder_state,
                        finish_logprob=node_type_logprob,
                        atom_choice_info=atom_choice_info,
                    )
                )
            else:
                new_decoder_state, added_motif = self._add_atom_or_motif(
                    decoder_state,
                    node_type_pick,
                    logprob=node_type_logprob,
                    choice_info=atom_choice_info,
                )

                if added_motif:
                    require_attachment_point_states.append(new_decoder_state)
                else:
                    require_bond_states.append(new_decoder_state)

    if self.uses_motifs:
        # Step 2': For states that require picking an attachment point, pick one:
        require_attachment_point_states = restrict_to_beam_size_per_mol(
            require_attachment_point_states, beam_size
        )
        (
            attachment_pick_results,
            attachment_pick_logits,
        ) = self._decoder_pick_attachment_points(
            decoder_states=require_attachment_point_states, sampling_mode=sampling_mode
        )

        for decoder_state, attachment_point_picks, attachment_point_logits in zip(
            require_attachment_point_states,
            attachment_pick_results,
            attachment_pick_logits,
        ):
            for (attachment_point_pick, attachment_point_logprob) in attachment_point_picks:
                attachment_point_choice_info = None
                if store_generation_traces:
                    attachment_point_choice_info = MoleculeGenerationAttachmentPointChoiceInfo(
                        partial_molecule_adjacency_lists=decoder_state.adjacency_lists,
                        motif_nodes=decoder_state.atoms_to_mark_as_visited,
                        candidate_attachment_points=decoder_state.candidate_attachment_points,
                        candidate_idx_to_prob=tf.nn.softmax(attachment_point_logits),
                        correct_attachment_point_idx=None,
                    )

                # print(I {decoder_state.molecule_id} {decoder_state.logprob:12f}: Picked attachment point {attachment_point_pick} - p={attachment_point_logprob:5f}")
                require_bond_states.append(
                    MoLeRDecoderState.new_with_focus_on_attachment_point(
                        decoder_state,
                        attachment_point_pick,
                        focus_atom_logprob=attachment_point_logprob,
                        attachment_point_choice_info=attachment_point_choice_info,
                    )
                )
    else:
        assert not require_attachment_point_states

    # Step 3: Pick fresh bonds and populate the next round of decoding steps:
    require_bond_states = restrict_to_beam_size_per_mol(require_bond_states, beam_size)
    bond_pick_results = self._decoder_pick_new_bond_types(
        decoder_states=require_bond_states,
        store_generation_traces=store_generation_traces,
        sampling_mode=sampling_mode,
    )
    for (decoder_state, (bond_picks, edge_choice_info)) in zip(
        require_bond_states, bond_pick_results
    ):
        if len(bond_picks) == 0:
            # There were no valid options for this bonds, so we treat this as if
            # predicting no more bonds with probability 1.0:
            # print(I {decoder_state.molecule_id} {decoder_state.logprob:12f}: No more allowed bonds to node {decoder_state.focus_atom}")
            new_decoder_states.append(
                MoLeRDecoderState.new_with_focus_marked_as_visited(
                    decoder_state,
                    focus_node_finished_logprob=0,
                    edge_choice_info=edge_choice_info,
                )
            )
            continue

        for (bond_pick, bond_pick_logprob) in bond_picks:
            # If the decoder says we need no more bonds for the current focus node,
            # we mark this and put the decoder state back for the next expansion round:
            if bond_pick is None:
                # print(I {decoder_state.molecule_id} {decoder_state.logprob:12f}: Finished connecting bonds to node {decoder_state.focus_atom} - p={bond_pick_logprob:5f}")
                new_decoder_states.append(
                    MoLeRDecoderState.new_with_focus_marked_as_visited(
                        decoder_state,
                        focus_node_finished_logprob=bond_pick_logprob,
                        edge_choice_info=edge_choice_info,
                    )
                )
            else:
                (picked_bond_target, picked_bond_type) = bond_pick

                # print(I {decoder_state.molecule_id} {decoder_state.logprob:12f}: Adding {decoder_state.focus_atom}-{picked_bond_type}->{picked_bond_target} - p={bond_pick_logprob:5f}")
                new_decoder_states.append(
                    MoLeRDecoderState.new_with_added_bond(
                        old_state=decoder_state,
                        target_atom_idx=int(
                            picked_bond_target
                        ),  # Go from np.int32 to pyInt
                        bond_type_idx=picked_bond_type,
                        bond_logprob=bond_pick_logprob,
                        edge_choice_info=edge_choice_info,
                    )
                )

    # Everything is done, restrict to the beam width, and go back to the loop start:
    decoder_states = restrict_to_beam_size_per_mol(new_decoder_states, beam_size)